In [1]:
import pandas as pd

# Import cleaned MetalliCan data

In [2]:
from utils.data_manipulations import build_activity_name, add_site_id

In [3]:
# Pre-processed production table
production_df = pd.read_excel(r'data/MetalliCan/sites_for_lci.xlsx', sheet_name='prod_data')

In [4]:
# Add activitiy_name to production_df and site_id
production_df['activity_name'] = production_df.apply(lambda row: build_activity_name(row, production_df), axis=1)
production_df = add_site_id(production_df)

In [5]:
# Normalized MetalliCan tables per ore processed
energy_ore_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/ore_normalization/energy_df.csv')
material_ore_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/ore_normalization/material_df.csv')
biosphere_ore_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/ore_normalization/biosphere_df.csv')

In [6]:
# Normalized MetalliCan tables per ore processed
energy_econ_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/economic_allocation/energy_df.csv')
material_econ_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/economic_allocation/material_df.csv')
biosphere_econ_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/economic_allocation/biosphere_df.csv')

In [7]:
# Removing rows with value_normalized is NaN in the biosphere dfs
biosphere_ore_df = biosphere_ore_df[~biosphere_ore_df['value_normalized'].isna()]
biosphere_econ_df['value_normalized'] = biosphere_econ_df['value_normalized'] / 1e6

In [8]:
# For all the dfs, multiply the value_normalized to 10e6 to have values per kg
energy_ore_df['value_normalized'] = energy_ore_df['value_normalized'] / 1e6
material_ore_df['value_normalized'] = material_ore_df['value_normalized'] / 1e6
biosphere_ore_df['value_normalized'] = biosphere_ore_df['value_normalized'] / 1e6
energy_econ_df['value_normalized'] = energy_econ_df['value_normalized'] / 1e6
material_econ_df['value_normalized'] = material_econ_df['value_normalized'] / 1e6
biosphere_econ_df['value_normalized'] = biosphere_econ_df['value_normalized'] / 1e6

# Keeping only relevant columns

In [9]:
technosphere_col = ['activity_name', 'functional_unit', 'site_id', 'subflow_type', 'value_normalized']
biosphere_col = ['activity_name', 'functional_unit', 'site_id', 'substance_name', 'value_normalized', 'unit']

In [10]:
energy_ore_df = energy_ore_df[technosphere_col]
material_ore_df = material_ore_df[technosphere_col]
biosphere_ore_df = biosphere_ore_df[biosphere_col]
energy_econ_df = energy_econ_df[technosphere_col]
material_econ_df = material_econ_df[technosphere_col]
biosphere_econ_df = biosphere_econ_df[biosphere_col]

In [11]:
# Add units to technosphere dfs
energy_ore_df['unit'] = 'MJ'
material_ore_df['unit'] = 't'
energy_econ_df['unit'] = 'MJ'
material_econ_df['unit'] = 't'

In [12]:
# Put energy_df and material_df together
technosphere_ore_df = pd.concat([energy_ore_df, material_ore_df], ignore_index=True)
technosphere_econ_df = pd.concat([energy_econ_df, material_econ_df], ignore_index=True)

In [13]:
# Add the province from the main_df to specify electricity location later
technosphere_ore_df = technosphere_ore_df.merge(production_df[['site_id', 'province']], on=['site_id'], how='left')
technosphere_econ_df = technosphere_econ_df.merge(production_df[['site_id', 'province']], on=['site_id'], how='left')
biosphere_ore_df = biosphere_ore_df.merge(production_df[['site_id', 'province']], on=['site_id'], how='left')
biosphere_econ_df = biosphere_econ_df.merge(production_df[['site_id', 'province']], on=['site_id'], how='left')

# Map MetalliCan flows to EI and RI flows

In [14]:
from utils.conversion_functions import map_technosphere_to_ecoinvent, map_biosphere_to_ecoinvent
from core.constants import CA_provinces

## Technosphere flows

In [15]:
mapping_technosphere = pd.read_excel(r'data/Mappings/MAPPINGS_RI.xlsx', sheet_name='technosphere')

In [16]:
technosphere_ore_df

,activity_name,functional_unit,site_id,subflow_type,value_normalized,unit,province
0,"Au, UG mining and beneficiation at Brucejack",Ore processed,BC-MAIN-857b7b89,Acetylene,1.112991e-07,MJ,British Columbia
1,"Au, UG mining and beneficiation at Brucejack",Ore processed,BC-MAIN-857b7b89,Aviation fuel,4.378079e-04,MJ,British Columbia
2,"Au, UG mining and beneficiation at Brucejack",Ore processed,BC-MAIN-857b7b89,Diesel,1.729171e-03,MJ,British Columbia
3,"Au, UG mining and beneficiation at Brucejack",Ore processed,BC-MAIN-857b7b89,Gasoline,8.173765e-05,MJ,British Columbia
4,"Au, UG mining and beneficiation at Brucejack",Ore processed,BC-MAIN-857b7b89,Propane,2.534400e-04,MJ,British Columbia
...,...,...,...,...,...,...,...
227,"Au, UG mining and beneficiation at Westwood-Doyon",Ore processed,QC-MAIN-02884fb5,Motor/drill oil,1.827853e-11,t,Quebec
228,"Au, UG mining and beneficiation at Westwood-Doyon",Ore processed,QC-MAIN-02884fb5,Sulfur dioxide,5.212766e-10,t,Quebec
229,"Au, UG mining and beneficiation at Westwood-Doyon",Ore processed,QC-MAIN-02884fb5,Transmission oil,7.833656e-12,t,Quebec
230,"Au, UG mining and beneficiation at Young-Davidson",Ore processed,ON-MAIN-7607a50e,Total blasting agents used e.g. ANFO,9.032167e-10,t,Ontario


In [17]:
# Apply the function
mapped_technosphere_ore_df = map_technosphere_to_ecoinvent(technosphere_ore_df, mapping_technosphere, CA_provinces)
mapped_technosphere_econ_df = map_technosphere_to_ecoinvent(technosphere_econ_df, mapping_technosphere, CA_provinces)

Index(['activity_name', 'functional_unit', 'site_id', 'subflow_type',
       'value_normalized', 'unit', 'province', 'Type', 'MetalliCan',
       'MetalliCan_unit', 'DB_to_map', 'Reference product', 'Flow name',
       'Location', 'Unit', 'Comment'],
      dtype='object')
⚠️ Les flux suivants n'ont pas trouvé de correspondance dans Ecoinvent:
 - Energy use
 - Other
 - Surface/underground emulsion & ANFO
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Acetylene)
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Used oil)
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Electricity consumption|Generated on-site)
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Electricity consumption|Generated on-site)
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Electricity consumption|Generated on-site)
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Electricity consumption|Non-renewable electricity use)
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Non

In [18]:
# Drop rows where ecoinvent_flow_name is "No mapping" and Amount is NaN for now
mapped_technosphere_ore_df = mapped_technosphere_ore_df[(mapped_technosphere_ore_df["Activity"] != "No mapping") & (~mapped_technosphere_ore_df["Amount"].isna())]
mapped_technosphere_econ_df = mapped_technosphere_econ_df[(mapped_technosphere_econ_df["Activity"] != "No mapping") & (~mapped_technosphere_econ_df["Amount"].isna())]

In [19]:
technosphere_col_for_lci = ['site_id', 'activity_name', 'functional_unit', 'Amount', 'Activity', 'Product', 'Unit', 'Location', 'Database']

In [20]:
mapped_technosphere_ore_df = mapped_technosphere_ore_df[technosphere_col_for_lci]
mapped_technosphere_econ_df = mapped_technosphere_econ_df[technosphere_col_for_lci]

## Biosphere flows mapping

In [21]:
mapping_biosphere = pd.read_excel(r'data/Mappings/MAPPINGS_RI.xlsx', sheet_name='biosphere')

In [22]:
mapped_biosphere_ore_df = map_biosphere_to_ecoinvent(biosphere_ore_df, mapping_biosphere, CA_provinces)
mapped_biosphere_econ_df = map_biosphere_to_ecoinvent(biosphere_econ_df, mapping_biosphere, CA_provinces)

Index(['activity_name', 'functional_unit', 'site_id', 'substance_name',
       'value_normalized', 'unit', 'province', 'Type', 'substance_id',
       'compartment_name', 'release_pathway', 'flow_direction',
       'MetalliCan_unit', 'DB_to_map', 'Flow name', 'Compartments', 'Unit',
       'Comment', 'Alternatives'],
      dtype='object')
⚠️ 6 biosphere flows could not be mapped to Ecoinvent:
   - 1-Nitropyrene
   - Quinoline
   - Ethylene glycol
   - Water
   - nan
   - PFCs
⚠️ No conversion defined for g teq → kilogram (flow: Dioxins and furans - total)
⚠️ No conversion defined for g teq → kilogram (flow: Dioxins and furans - total)
⚠️ No conversion defined for kg → nan (flow: 1-Nitropyrene)
⚠️ No conversion defined for g teq → kilogram (flow: Dioxins and furans - total)
⚠️ No conversion defined for g teq → kilogram (flow: Dioxins and furans - total)
⚠️ No conversion defined for grams → kilogram (flow: Hexachlorobenzene)
⚠️ No conversion defined for grams → kilogram (flow: Hexachlorob

In [23]:
# Drop rows where ecoinvent_flow_name is "No mapping" and Amount is NaN for now
mapped_biosphere_ore_df = mapped_biosphere_ore_df[(mapped_biosphere_ore_df["Flow Name"] != "No mapping") & (~mapped_biosphere_ore_df["Amount"].isna())]
mapped_biosphere_econ_df = mapped_biosphere_econ_df[(mapped_biosphere_econ_df["Flow Name"] != "No mapping") & (~mapped_biosphere_econ_df["Amount"].isna())]

In [24]:
biosphere_col_for_lci = ['site_id', 'activity_name', 'functional_unit', 'Amount', 'Unit', 'Flow Name', 'Compartments', 'Database']

In [25]:
mapped_biosphere_ore_df = mapped_biosphere_ore_df[biosphere_col_for_lci]
mapped_biosphere_econ_df = mapped_biosphere_econ_df[biosphere_col_for_lci]

In [26]:
production_df.to_csv(r'data/MetalliCan/data_for_lci_initialization/production_df.csv', index=False)

In [27]:
mapped_biosphere_ore_df.to_csv(r'data/MetalliCan/data_for_lci_initialization/biosphere_ore_df.csv', index=False)
mapped_biosphere_econ_df.to_csv(r'data/MetalliCan/data_for_lci_initialization/biosphere_econ_df.csv', index=False)

In [28]:
mapped_technosphere_ore_df.to_csv(r'data/MetalliCan/data_for_lci_initialization/technosphere_ore_df.csv', index=False)
mapped_technosphere_econ_df.to_csv(r'data/MetalliCan/data_for_lci_initialization/technosphere_econ_df.csv', index=False)

In [29]:
# Add province and commodities from NRCan to production table
mapped_biosphere_ore_df = mapped_biosphere_ore_df.merge(production_df[['site_id', 'province', 'commodities']], on=['site_id'], how='left')
mapped_biosphere_econ_df = mapped_biosphere_econ_df.merge(production_df[['site_id', 'province', 'commodities']], on=['site_id'], how='left')

In [30]:
mapped_technosphere_econ_df

,site_id,activity_name,functional_unit,Amount,Activity,Product,Unit,Location,Database
0,BC-MAIN-857b7b89,"Au, UG mining and beneficiation at Brucejack","Au, usable ore",0.020483,consumption market for acetylene,acetylene,kilogram,CA,Regioinvent
1,BC-MAIN-857b7b89,"Au, UG mining and beneficiation at Brucejack","Au, usable ore",1.869421,consumption market for kerosene,kerosene,kilogram,CA,Regioinvent
2,BC-MAIN-857b7b89,"Au, UG mining and beneficiation at Brucejack","Au, usable ore",7.400659,consumption market for diesel,diesel,kilogram,CA,Regioinvent
3,BC-MAIN-857b7b89,"Au, UG mining and beneficiation at Brucejack","Au, usable ore",0.338797,consumption market for diesel,diesel,kilogram,CA,Regioinvent
4,BC-MAIN-857b7b89,"Au, UG mining and beneficiation at Brucejack","Au, usable ore",1.005213,consumption market for propane,propane,kilogram,CA,Regioinvent
...,...,...,...,...,...,...,...,...,...
320,QC-MAIN-02884fb5,"Au, UG mining and beneficiation at Westwood-Doyon","Au, usable ore",0.186336,"consumption market for sulfur dioxide, liquid","sulfur dioxide, liquid",kilogram,CA,Regioinvent
321,QC-MAIN-02884fb5,"Au, UG mining and beneficiation at Westwood-Doyon","Au, usable ore",0.002800,consumption market for lubricating oil,lubricating oil,kilogram,CA,Regioinvent
322,ON-MAIN-7607a50e,"Au, UG mining and beneficiation at Young-Davidson","Au, usable ore",0.451517,"consumption market for explosive, tovex","explosive, tovex",kilogram,CA,Regioinvent
323,ON-MAIN-7607a50e,"Au, UG mining and beneficiation at Young-Davidson","Au, usable ore",0.451517,"consumption market for explosive, tovex","explosive, tovex",kilogram,CA,Regioinvent


# LCI creation

In [31]:
from core.lci_database_builder import LCIDatabaseBuilder

## Ore processed basism

In [38]:
# Step 1 — initialize the builder
builder_op = LCIDatabaseBuilder(db_name='metallican_lci', project_name='metallican')

📂 Active Brightway project: metallican
🆕 Database 'metallican_lci' created.


In [39]:
# Step 2 — create the activity shells from the main dataframe
builder_op.build_lci_entries(df=mapped_biosphere_ore_df)
print(len(builder_op.lcis))

✅ Created 28 base LCI activities with production exchanges.
28


In [40]:
# Step 3a — Populate with the technosphere exchanges
builder_op.populate_technosphere_exchanges(technosphere_df=mapped_technosphere_ore_df)

⚙️ Populating technosphere exchanges
   ✅ Cached 218246 activities from Regioinvent
   ✅ Cached 20769 activities from ecoinvent-3.10-cutoff regionalized
✅ Added 206 technosphere exchanges.


In [41]:
# Step 3b — Populate with the biosphere exchanges
builder_op.populate_biosphere_exchanges(biosphere_df=mapped_biosphere_ore_df)

🌱 Populating biosphere exchanges
   ✅ Cached 110559 biosphere flows from biosphere3_spatialized_flows
   ✅ Cached 4362 biosphere flows from biosphere3
✅ Added 5817 biosphere exchanges.
⚠️ 28 biosphere flows could not be matched:
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   ... and 18 more.


In [42]:
# Step 4 - Consolidate duplicate flows
builder_op.consolidate_exchanges()

🧮 Consolidation: 6051 → 663 exchanges (summed duplicates).


In [43]:
builder_op.write_to_database()

🧱 Writing 28 activities to database 'metallican_lci'...
✅ Database 'metallican_lci' processed successfully with 28 activities.


## Usable ore basis

In [44]:
# Step 1 — initialize the builder
builder_uo = LCIDatabaseBuilder(db_name='metallican_lci', project_name='metallican')

📂 Active Brightway project: metallican
✅ Using existing database 'metallican_lci'.


In [45]:
# Step 2 — create the activity shells from the main dataframe
builder_uo.build_lci_entries(df=mapped_biosphere_econ_df)
print(len(builder_uo.lcis))

✅ Created 45 base LCI activities with production exchanges.
45


In [34]:
# Step 3a — Populate with the technosphere exchanges
builder_uo.populate_technosphere_exchanges(technosphere_df=mapped_technosphere_econ_df)

⚙️ Populating technosphere exchanges
   ✅ Cached 218246 activities from Regioinvent
   ✅ Cached 20769 activities from ecoinvent-3.10-cutoff regionalized
✅ Added 307 technosphere exchanges.


In [46]:
# Step 3b — Populate with the biosphere exchanges
builder_uo.populate_biosphere_exchanges(biosphere_df=mapped_biosphere_econ_df)

🌱 Populating biosphere exchanges
   ✅ Cached 110559 biosphere flows from biosphere3_spatialized_flows
   ✅ Cached 4362 biosphere flows from biosphere3
✅ Added 9128 biosphere exchanges.
⚠️ 41 biosphere flows could not be matched:
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   - ('soil',) (emission, biosphere3)
   ... and 31 more.


In [47]:
# Step 4 - Consolidate duplicate flows
builder_uo.consolidate_exchanges()

🧮 Consolidation: 9173 → 828 exchanges (summed duplicates).


In [48]:
builder_uo.write_to_database()

🧱 Writing 45 activities to database 'metallican_lci'...
✅ Database 'metallican_lci' processed successfully with 73 activities.
